In [6]:
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

N = 10000 # number of samples per file
D = 3072 # number of features (dimensions of each picture)
K = 10 # number of classes

In [2]:
def LoadBatch(filename):
	""" Copied from the dataset website """
	import pickle
	with open(filename, 'rb') as fo:
		dict = pickle.load(fo, encoding='bytes')
	return dict

def montage(W):
	""" Display the image for each label in W """
	fig, ax = plt.subplots(2,5)
	for i in range(2):
		for j in range(5):
			im  = W[i*5+j,:].reshape(32,32,3, order='F')
			sim = (im-np.min(im[:]))/(np.max(im[:])-np.min(im[:]))
			sim = sim.transpose(1,0,2)
			ax[i][j].imshow(sim, interpolation='nearest')
			ax[i][j].set_title("y="+str(5*i+j))
			ax[i][j].axis('off')
	plt.show()

def loader(filepath):

    dic = LoadBatch("Datasets\\" + filepath)
    data = dic[b'data'].T # shape = d x n = 3072 x 10000
    labels = np.array(dic[b'labels']).flatten() # shape = n = 1000

    y = np.zeros((K, N))
    for i in range(N):
        y[labels[i]][i] = 1

    return data, y, labels

def extended_loader():
    Data = []
    OneHot = []
    Labels = []
    for i in range(5):
       D, H, L = loader(f"data_batch_{i+1}")
       Data.append(D)
       OneHot.append(H)
       Labels.append(L)
    return {"Data" :np.hstack(Data), "OneHot": np.hstack(np.array(OneHot)),"Labels" : np.hstack(Labels)}

def val_split(data,val_per=0.02):
    n_val = val_per * data["Labels"].shape[0]
    split = int(data["Labels"].shape[0] - n_val)
    train = {"Data" :data["Data"][:, :split], "OneHot": data["OneHot"][:, :split],"Labels" : data["Labels"][:split]}
    val = {"Data" :data["Data"][:, split:], "OneHot": data["OneHot"][:, split:],"Labels" : data["Labels"][split:]}
    return train, val

def mean_std(train_X):
    mean = np.mean(train_X, axis=1, keepdims=True)
    std = np.std(train_X, axis=1, keepdims=True)
    return mean, std

def normalize(mean, std, X):
    norm_X = (X - mean) / std
    return norm_X



#### Loading the Dataset


In [3]:
#load and split
Data = extended_loader()

train_X, train_Y, train_y = Data.values() # Data, one hot labels, labels
test_X, test_Y, test_y = loader("test_batch")

print('Train: X=%s, y=%s' % (train_X.shape, train_Y.shape))
print('Test: X=%s, y=%s' % (test_X.shape, test_Y.shape))

Train: X=(3072, 50000), y=(10, 50000)
Test: X=(3072, 10000), y=(10, 10000)


In [17]:
train_X = torch.tensor(train_X)
test_X = torch.tensor(test_X)
train_Y = torch.tensor(train_Y) 
test_Y = torch.tensor(test_Y) 

#### Normalizing

In [18]:
train_X_norm = train_X / 255.0
test_X_norm = test_X / 255.0

#### Setting up the model

In [22]:
class Cifar10Model(nn.Module):
    def __init__(self):
        super(Cifar10Model,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), padding=1),
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), padding=1),
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3,3), padding=1),
        self.conv4 = nn.Conv2d(64, 64, kernel_size=(3,3), padding=1),
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=1),
        self.conv6 = nn.Conv2d(128, 128, kernel_size=(3,3), padding=1),
        
        self.pool = nn.MaxPool2d(2, 2)
    
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool1(x)
        
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool2(x)
        
        x = self.relu(self.conv5(x))
        x = self.relu(self.conv6(x))
        x = self.pool3(x)
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

In [21]:
model = Cifar10Model()
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)